In [1]:
import os 
from config import settings
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

In [2]:

NEO4J_URI = settings.NEO4J_URI
NEO4J_USERNAME = settings.NEO4J_USERNAME
NEO4J_PASSWORD = settings.NEO4J_PASSWORD
OPENAI_API_KEY = settings.OPENAI_API_KEY
OPENAI_ENDPOINT = settings.OPENAI_ENDPOINT or "https://api.openai.com/v1"
TMDB_API_KEY = settings.TMDB_API_KEY
OMDB_API = settings.OMDB_API

In [3]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
response= llm.invoke("What is NEO4j")

print(response)



NEO4j is a graph database management system designed to store, manage, and query highly connected data. It is based on the concept of a graph, which consists of nodes (representing entities) and relationships (representing connections between entities). These nodes and relationships can have properties associated with them, making it a flexible and powerful tool for modeling complex and interconnected data. It uses the Cypher query language, which allows for intuitive and efficient querying of data in the graph. NEO4j is used in various industries, including finance, healthcare, and social media, for tasks such as fraud detection, recommendation engines, and network analysis. 


In [14]:
template = PromptTemplate(template="""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""", input_variables={"fruit"})

response = llm.invoke(template.format(fruit="apple"))

print(response)


Well, mate, that's a right juicy Adam and Eve you got there. Perfect for a nice pie or crumble, innit?


In [15]:
#Configuring the LLM
llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-3.5-turbo-instruct",
    temperature = 0
    )

In [19]:
## chaining


template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

llm_chain = template | llm # this is called chaining

response = llm_chain.invoke({"fruit":"apple"})

#another way of chaining
#response = llm.invoke(template.format(fruit="apple"))
print(response)


Well, mate, that's a right juicy Adam and Eve you got there. Perfect for a nice pie or crumble, innit?


In [23]:
# adding output parser

from langchain.schema import StrOutputParser

template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Output JSON as {{"description": "your response here"}}

Tell me about the following fruit: {fruit}
""")
llm_chain = template | llm | StrOutputParser()
response = llm_chain.invoke({"fruit":"apple"})
print(response)


{"type": "Apples and pears, mate. They're the bee's knees of fruit. Crunchy and juicy, they are!"}


## Chat Models

### Create a Chat Model

A language model predicts the next word in a sequence of words. Chat models are designed to have conversations - they accept a list of messages and return a conversational response.

In [29]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage 
chat_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY) #see the difference in the object function

instructions = SystemMessage(content="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.
""")

question = HumanMessage(content="What is the weather like?")
response = chat_llm.invoke([
    instructions,
    question
])

print(response.content)


Dude, the weather is totally gnarly today! The sun is shining, the waves are pumping, and the vibes are just totally epic. It's going to be a sick day out on the water, for sure! Are you hitting the waves today too, bro?


### Wrapping Chain

In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


chat_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        (
            "human", 
            "{question}"
        ),
    ]
)


chat_chain = prompt | chat_llm | StrOutputParser()

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)

Dude, the weather is epic! Clear skies, offshore winds, and the waves are firing! It's gonna be a sick sesh out there today! Stoked to hit the waves, brah!


### Giving context

In [5]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

chat_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ( "system", "{context}" ),
        ( "human", "{question}" ),
    ]
)

chat_chain = prompt | chat_llm | StrOutputParser()

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

response = chat_chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like on Watergate Bay?",
    }
)

print(response)

Dude, Watergate Bay is firing with 3ft waves right now, but the wind is a bit of a bummer with onshore winds.


### Giving a Chat Model Memory

In [6]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

memory = ChatMessageHistory()

def get_memory(session_id):
    return memory

chat_chain = prompt | chat_llm | StrOutputParser()

#giving history

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)


current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while (question := input("> ")) != "exit":

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": "none"}
        }
    )
    
    print(response)

>  how is the wether in Ireland


Dude, the surf at Fistral is firing! We've got some sick 6ft waves and offshore winds, it's gonna be epic out there! If you're looking for some mellow vibes, maybe head to Bells - it's flat and calm over there. Or if you're up for a challenge, Watergate Bay is pumping with 3ft waves and onshore winds. Choose your beach wisely, bro!


>  exit


In [7]:
## session id 
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: 2f7cd6a0-0808-4994-81d3-27c020adb17c


### Connecting to the NEO4J

In [21]:
import os
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

result = graph.query("""
MATCH (m:Movie {title: 'Four Rooms'})
RETURN m.tagline

""")

print(result)

[{'m.tagline': "Twelve outrageous guests. Four scandalous requests. And one lone bellhop, in his first day on the job, who's in for the wildest New year's Eve of his life."}]


In [22]:
print(graph.schema)

Node properties:
Movie {title: STRING, tagline: STRING, taglineEmbedding: LIST, tmdb_id: INTEGER, overview: STRING, release_date: DATE, poster_path: STRING, poster_url: STRING, trailer_url: STRING, popularity: FLOAT, vote_average: FLOAT, vote_count: INTEGER, last_updated: DATE_TIME, poster_embedding: LIST}
Person {name: STRING, tmdb_id: INTEGER, gender: INTEGER, last_updated: DATE_TIME}
Relationship properties:
ACTED_IN {roles: LIST}
The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)


## Adding chat History using session_ID

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import Neo4jChatMessageHistory
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

chat_llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while (question := input("> ")) != "exit":
    
    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": SESSION_ID}
        }
    )
    
    print(response)

Session ID: 7c8d7dd9-a49d-42b1-8ccb-277e8778c1bf
